# Импорт библиотек и константы

In [1]:
# !pip install transformers --quiet 
# !pip install pandas numpy nltk gdown

In [2]:
# !sudo apt install zip -y

In [3]:
!pip install datasets seqeval transformers --quiet

     |████████████████████████████████| 325 kB 20.9 MB/s 
     |████████████████████████████████| 43 kB 417 kB/s 
     |████████████████████████████████| 4.0 MB 42.7 MB/s 
     |████████████████████████████████| 77 kB 3.2 MB/s 
     |████████████████████████████████| 212 kB 52.4 MB/s 
     |████████████████████████████████| 136 kB 52.5 MB/s 
     |████████████████████████████████| 1.1 MB 28.9 MB/s 
     |████████████████████████████████| 127 kB 52.8 MB/s 
     |████████████████████████████████| 895 kB 22.7 MB/s 
     |████████████████████████████████| 6.6 MB 49.4 MB/s 
     |████████████████████████████████| 596 kB 52.1 MB/s 
     |████████████████████████████████| 144 kB 57.1 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 73.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 

In [4]:
!git clone https://github.com/tatnashev/prozhito

Cloning into 'prozhito'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 43 (delta 18), reused 37 (delta 15), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [5]:
import pandas as pd 
import numpy as np 
import random
from collections import defaultdict, Counter
import nltk 
import re

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import torch 
from torch import nn 
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader 
from torch.optim import Adam
from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModelForPreTraining
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset, DatasetDict, load_metric
import torch.nn.functional as F

In [8]:
# from google.colab import drive 

# drive.mount('/content/drive')

"Константы"

In [9]:
label_list = ['B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-FAC', 'I-FAC', 'B-PER', 'I-PER', 'B-CHAR', 'I-CHAR', 'O']
TOKENIZER_PATH = "DeepPavlov/rubert-base-cased"
MODEL_PATH = "DeepPavlov/rubert-base-cased"
OUTPUT_DIR = '.'
res = {}
tokenizer = None 

In [10]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [11]:
set_random_seed(42)

## Подготовка текстов Прожито 

In [12]:
# download prozhito texts 
import gdown

url = 'https://drive.google.com/file/d/1oHAmsa6Nxb1B2sbWriJXCExETwSirtMj/view?usp=sharing' 
gdown.download(url, 'prozhito_texts.zip', fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1oHAmsa6Nxb1B2sbWriJXCExETwSirtMj
To: /content/prozhito_texts.zip
100%|██████████| 20.8M/20.8M [00:00<00:00, 36.1MB/s]


'prozhito_texts.zip'

In [13]:
!unzip prozhito_texts.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: prozhito_texts/457/122178.txt  
  inflating: prozhito_texts/457/122179.txt  
 extracting: prozhito_texts/457/122195.ann  
  inflating: prozhito_texts/457/122186.txt  
  inflating: prozhito_texts/457/122192.txt  
 extracting: prozhito_texts/457/122181.ann  
 extracting: prozhito_texts/457/122208.ann  
 extracting: prozhito_texts/457/122220.ann  
  inflating: prozhito_texts/457/122233.txt  
  inflating: prozhito_texts/457/122227.txt  
 extracting: prozhito_texts/457/122234.ann  
  inflating: prozhito_texts/457/122231.txt  
 extracting: prozhito_texts/457/122222.ann  
 extracting: prozhito_texts/457/122236.ann  
  inflating: prozhito_texts/457/122225.txt  
  inflating: prozhito_texts/457/122219.txt  
  inflating: prozhito_texts/457/122184.txt  
 extracting: prozhito_texts/457/122197.ann  
 extracting: prozhito_texts/457/122183.ann  
  inflating: prozhito_texts/457/122190.txt  
  inflating: prozhito_texts/457/

In [14]:
import glob, os

sentences = [] 
bio_nums = [] 
rootdir = 'prozhito_texts'
for subdir, dirs, files in os.walk(rootdir):
    for filename in files:
        if filename.endswith('.txt'): 
            path = os.path.join(subdir, filename)
            with open(path) as f: 
                sents = f.readlines() 
                for sentence in sents: 
                    sentences.append(sentence.strip())   
                    bio_nums.append([10] * len([word for word in nltk.word_tokenize(sentences[-1])]))

In [15]:
sentences = sentences
bio_nums = bio_nums

In [16]:
len(sentences)

10

In [17]:
len(bio_nums)

10

In [18]:
bio_nums[0]

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

# Загрузка данных 

In [19]:
def preprocess_labels(labels): 
    nums = labels.split(', ')
    nums[0] = nums[0][1:] 
    nums[-1] = nums[-1][:-1]

    return list(map(int, nums))

In [20]:
df_train = pd.read_csv('prozhito/prozhito_data/df_train_prozhito.csv')
df_test = pd.read_csv('prozhito/prozhito_data/df_test_prozhito.csv')
df_val = pd.read_csv('prozhito/prozhito_data/df_val_prozhito.csv')

In [21]:
df_train['BIO_nums'] = df_train['BIO_nums'].apply(lambda x: preprocess_labels(x))
df_test['BIO_nums'] = df_test['BIO_nums'].apply(lambda x: preprocess_labels(x))
df_val['BIO_nums'] = df_val['BIO_nums'].apply(lambda x: preprocess_labels(x))

df_train.head() 

,Unnamed: 0,tokens,BIO_str,BIO_nums,BIO_list
0,0,У меня большая симпатия к Лукьянину — человек ...,O O O O O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,2,"> Каким приговором , указом каким > Ты здесь ,...",O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,3,"Подумал , что летом ребята куда затащили .",O O O O B-CHAR O O O,"[10, 10, 10, 10, 8, 10, 10, 10]","['O', 'O', 'O', 'O', 'B-CHAR', 'O', 'O', 'O']"
3,4,Нашел потрясающие материалы о В . М . Брадисе ...,O O O O B-PER I-PER I-PER I-PER I-PER O O B-CH...,"[10, 10, 10, 10, 4, 5, 5, 5, 5, 10, 10, 8, 8, ...","['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER'..."
4,5,[Без даты . ],O O O O,"[10, 10, 10, 10]","['O', 'O', 'O', 'O']"


In [22]:
df_train = pd.concat([df_train, df_test], ignore_index=True)
df_train = df_train.drop(columns=['Unnamed: 0'])
df_train.head()

,tokens,BIO_str,BIO_nums,BIO_list
0,У меня большая симпатия к Лукьянину — человек ...,O O O O O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"> Каким приговором , указом каким > Ты здесь ,...",O O O O O O O O O O O O O O O O O,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"Подумал , что летом ребята куда затащили .",O O O O B-CHAR O O O,"[10, 10, 10, 10, 8, 10, 10, 10]","['O', 'O', 'O', 'O', 'B-CHAR', 'O', 'O', 'O']"
3,Нашел потрясающие материалы о В . М . Брадисе ...,O O O O B-PER I-PER I-PER I-PER I-PER O O B-CH...,"[10, 10, 10, 10, 4, 5, 5, 5, 5, 10, 10, 8, 8, ...","['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER'..."
4,[Без даты . ],O O O O,"[10, 10, 10, 10]","['O', 'O', 'O', 'O']"


In [23]:
df_test = pd.DataFrame({
    'tokens': sentences, 
    'BIO_nums': bio_nums
})

df_test.head()

,tokens,BIO_nums
0,"21-го октября состоялся пленум ЦК , на котором...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
1,"Когда доклад был потом сделан , он разочаровал...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]"
2,"Ожидали , что история этих семидесяти лет буде...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
3,"В первую очередь , с полным , не ретушированны...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
4,"Во вторую , со столь же полной реабилитацией Т...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."


In [24]:
df_test.shape

(10, 2)

In [25]:
# Считаем длину последовательности, 
# это понадобится дальше 

df_train['length'] = df_train['BIO_list'].apply(lambda x: len(x))
df_test['length'] = df_test['BIO_nums'].apply(lambda x: len(x))
df_val['length'] = df_val['BIO_nums'].apply(lambda x: len(x))

In [26]:
data = DatasetDict({
    'train': Dataset.from_pandas(df_train[['tokens', 'BIO_nums', 'length']]),
    'test': Dataset.from_pandas(df_test[['tokens', 'BIO_nums', 'length']]),
    'val': Dataset.from_pandas(df_val[['tokens', 'BIO_nums', 'length']])
})

data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 12
    })
    test: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 10
    })
    val: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 4
    })
})

In [27]:
# data = data.remove_columns('__index_level_0__')
# data

# Подготовка данных 

In [28]:
def tokenizer_and_align(texts): 
    tokenized_input = tokenizer(texts['tokens'], truncation=True)

    labels = []
    for i, label in enumerate(texts['BIO_nums']): 
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids: 
            if word_idx is None: 
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                if word_idx > len(label) - 1: 
                    label_ids.append(10)
                else: 
                    label_ids.append(label[word_idx])
            else: 
                if word_idx > len(label) - 1: 
                    label_ids.append(10)
                else:
                    label_ids.append(label[previous_word_idx])

            previous_word_idx = word_idx
            

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]
        labels.append(label_ids)

    tokenized_input['labels'] = labels 
    return tokenized_input 

In [29]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [30]:
tokenized_datasets = data.map(tokenizer_and_align, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('tokens')

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [31]:
cnt = 0
idxs = []
for i, bio in enumerate(tokenized_datasets['train']['BIO_nums']): 
    prev = None 
    for elem in bio: 
        if prev is not None and elem == prev and not elem % 2 and elem != 10: 
            cnt += 1
            idxs.append(i) 
        prev = elem

cnt

2

## Сортировка по длине батчей

In [32]:
tokenized_datasets = tokenized_datasets.sort('length')

In [33]:
tokenized_datasets = tokenized_datasets.remove_columns('length')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['BIO_nums', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 12
    })
    test: Dataset({
        features: ['BIO_nums', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    val: Dataset({
        features: ['BIO_nums', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})

# Обучение

In [34]:
def get_key_to_results(model_path, epochs, lr, weight_decay): 
    model = model_path.split('/')[-1]
    k = f'{model}, epochs = {epochs}, lr = {lr}, weight_decay = {weight_decay}'
    return k 

In [35]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

In [36]:
# model = model.to(device)

In [37]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [38]:
metric = load_metric("seqeval")

In [39]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [40]:
BIO_weights = [200.35416666666669,
            1479.5384615384614,
            1373.857142857143,
            1282.2666666666667,
            2323.4,
            10017.0,
            89.46046511627908,
            164.3931623931624,
            406.81720430107526,
            10,
            1.0308162280936812]

In [41]:
normedWeights = torch.FloatTensor(BIO_weights).cuda()

class MyTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        
        if "labels" in inputs:
          labels = inputs.pop("labels")
        
        outputs = model(**inputs)
        logits = outputs.logits
        # print('logits', logits.shape)
        # print('logits', logits.permute(0, 2, 1).shape)
        logits = logits.permute(0, 2, 1)
        # print('labels', labels.shape)
        loss_function = CrossEntropyLoss(weight = normedWeights)

        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = loss_function(logits, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss

In [42]:
def predict(model, trainer): 
    predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    
    # to dataframe 
    domains_ans = []
    f1s = []
    numbers = []
    precs = []
    recalls = []

    for key in sorted(results.keys()):
        if 'overall' not in key:
            domains_ans.append(key)
            f1s.append(round(results[key]['f1'], 4))
            numbers.append(round(results[key]['number'], 4))
            precs.append(round(results[key]['precision'], 4))
            recalls.append(round(results[key]['recall'], 4))
        else:
            if key == 'overall_f1': 
                f1s.append(round(results[key], 4))
            elif key == 'overall_precision': 
                precs.append(round(results[key], 4)) 
            elif key == 'overall_recall': 
                recalls.append(round(results[key], 4))

    domains_ans.append('Overall')
    numbers.append(sum(numbers))

    to_add = pd.DataFrame({'tag': domains_ans,
                'f1': f1s,
                'precision': precs,
                'recall': recalls,
                'number of occurence': numbers})
    
    return to_add, predictions


In [43]:
def train(model, config): 
    epochs = config['epochs']
    lr = config['lr']
    weight_decay = config['weight_decay']
    batch_size = config['bs']

    args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        evaluation_strategy = "epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        report_to='none',
        save_strategy='no', 
        # load_best_model_at_end=True, 
        logging_dir=OUTPUT_DIR,
        logging_strategy='epoch'
    )

    trainer = MyTrainer(
        model,
        args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['val'],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    for param in model.parameters():
        param.requires_grad = True

    import logging
    from transformers.trainer import logger as noisy_logger
    noisy_logger.setLevel(logging.WARNING)

    trainer.train() 

    return trainer 


In [44]:
configs = [
    {
        'epochs': 5, 
        'lr': 2e-5, 
        'weight_decay': 1e-4, 
        'bs': 8
    }
]

In [45]:
histories = [] 

In [46]:
config = configs[0]

trainer = train(model, config)
# to_add, predictions = predict(model, trainer)
# new_key = get_key_to_results(MODEL_PATH, config['epochs'], config['lr'], config['weight_decay'])
# res[new_key] = to_add 
# histories.append(trainer.state.log_history)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,2.064700,2.488531,0.000000,0.000000,0.000000,0.036585
2,1.981100,2.587994,0.000000,0.000000,0.000000,0.000000
3,0.754300,2.674191,0.000000,0.000000,0.000000,0.000000
4,0.609000,2.739656,0.000000,0.000000,0.000000,0.000000
5,1.195900,2.751215,0.000000,0.000000,0.000000,0.000000


## Достаем предсказания 

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [48]:
model = model.to(device) 

In [49]:
def get_preds(text): 
    inputs = tokenizer(text, return_tensors='pt')
    tokens = inputs.tokens()
    inputs = inputs.to(device)

    outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)

    preds = []
    for pred in predictions.cpu().numpy()[0]:
        preds.append(pred)

    return preds

In [50]:
data = DatasetDict({
    'train': Dataset.from_pandas(df_train[['tokens', 'BIO_nums', 'length']]),
    'test': Dataset.from_pandas(df_test[['tokens', 'BIO_nums', 'length']]),
    'val': Dataset.from_pandas(df_val[['tokens', 'BIO_nums', 'length']])
})

data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 12
    })
    test: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 10
    })
    val: Dataset({
        features: ['tokens', 'BIO_nums', 'length'],
        num_rows: 4
    })
})

In [51]:
tokenized_datasets = data.map(tokenizer_and_align, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
!pip install tqdm 

In [52]:
from tqdm import tqdm 

true_labels = []
predictions = []

for elem in tqdm(tokenized_datasets['test']): 
    text, labels = elem['tokens'], elem['labels']
    preds = get_preds(text)

    true_labels.append(labels)
    predictions.append(preds)
    if len(labels) != len(preds): 
        print(f'Oh no, len labels: {len(labels)}, len preds: {len(preds)}')

In [53]:
df_test_res = pd.DataFrame({
    'text': df_test['tokens'].values, 
    'labels': true_labels, 
    'preds': predictions
})

df_test_res.head(10)

,text,labels,preds
0,"21-го октября состоялся пленум ЦК , на котором...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
1,"Когда доклад был потом сделан , он разочаровал...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]"
2,"Ожидали , что история этих семидесяти лет буде...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
3,"В первую очередь , с полным , не ретушированны...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
4,"Во вторую , со столь же полной реабилитацией Т...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
5,"Наконец , ждали откровенной оценки ситуации с ...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
6,"Предполагали , что в своем докладе Горбачев бр...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
7,"Это было наивно , так как традиция праздничной...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
8,"Кроме того , как стало известно от нечиновных ...","[-100, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,...","[7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
9,Горбачев проиграл .,"[-100, 10, 10, 10, -100]","[7, 5, 5, 5, 3]"


In [54]:
def replace_nums_labels(values): 
    return [x for x in values if x!=-100]

df_test_res['labels'] = df_test_res['labels'].apply(lambda x: replace_nums_labels(x))

In [55]:
def replace_nums_preds(values): 
    return values[1:-1]

df_test_res['preds'] = df_test_res['preds'].apply(lambda x: replace_nums_preds(x))

In [56]:
tag_BIO = {v: k for k, v in model.config.label2id.items()}
tag_BIO

{0: 'B-LOC',
 1: 'I-LOC',
 2: 'B-ORG',
 3: 'I-ORG',
 4: 'B-FAC',
 5: 'I-FAC',
 6: 'B-PER',
 7: 'I-PER',
 8: 'B-CHAR',
 9: 'I-CHAR',
 10: 'O'}

In [57]:
def change_nums_to_tag(values): 
    return [tag_BIO[x] for x in values]


df_test_res['labels'] = df_test_res['labels'].apply(lambda x: change_nums_to_tag(x))
df_test_res['preds'] = df_test_res['preds'].apply(lambda x: change_nums_to_tag(x))

In [58]:
df_test_res.head(10)

,text,labels,preds
0,"21-го октября состоялся пленум ЦК , на котором...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
1,"Когда доклад был потом сделан , он разочаровал...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
2,"Ожидали , что история этих семидесяти лет буде...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
3,"В первую очередь , с полным , не ретушированны...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
4,"Во вторую , со столь же полной реабилитацией Т...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
5,"Наконец , ждали откровенной оценки ситуации с ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
6,"Предполагали , что в своем докладе Горбачев бр...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
7,"Это было наивно , так как традиция праздничной...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
8,"Кроме того , как стало известно от нечиновных ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-FAC, I-F..."
9,Горбачев проиграл .,"[O, O, O]","[I-FAC, I-FAC, I-FAC]"


In [59]:
df_test_res.to_csv('all_prozhito_texts_markup.csv')